<a href="https://colab.research.google.com/github/arminsoltan/mohaymen/blob/main/Phase02/CIFAR_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import torch
import torch.nn as nn
import numpy as np
import torchvision
from torchvision import transforms
import warnings
import torch.nn.functional as F

In [13]:
warnings.filterwarnings("ignore")

In [14]:
# Hyper-parameters
num_epochs = 5
batch_size = 4
learning_rate = 0.001

In [15]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified


In [16]:
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
  
  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16 * 5 * 5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [19]:
model = ConvNet()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    # Forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 2000 == 0:
      print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print("Finished Training")
PATH='./cnn.pth'
torch.save(model.state_dict(), PATH)

with torch.no_grad():
  n_correct = 0
  n_samples = 0
  n_class_correct = [0 for i in range(10)]
  n_class_samples = [0 for i in range(10)]
  for images, labels in test_loader:
    outputs = model(images)

    _, predicted = torch.max(outputs, 1)
    n_samples += labels.size(0)
    n_correct += (predicted == labels).sum().item()

    for i in range(batch_size):
      label = labels[i]
      pred = predicted[i]
      if (label == pred):
        n_class_correct[label] += 1
      n_class_samples[label] += 1
  
  acc = 100.0 * n_correct / n_samples
  print(f'Accuracy of the network: {acc} %')

  for i in range(10):
    acc = 100.0 * n_class_correct[i] / n_class_samples[i]
    print(f'Accuracy of {classes[i]}:{acc} %')


Epoch [1/5], Step [2000/12500], Loss: 2.2984
Epoch [1/5], Step [4000/12500], Loss: 2.3037
Epoch [1/5], Step [6000/12500], Loss: 2.2825
Epoch [1/5], Step [8000/12500], Loss: 2.3089
Epoch [1/5], Step [10000/12500], Loss: 2.2693
Epoch [1/5], Step [12000/12500], Loss: 2.2771
Epoch [2/5], Step [2000/12500], Loss: 2.2688
Epoch [2/5], Step [4000/12500], Loss: 2.2054
Epoch [2/5], Step [6000/12500], Loss: 1.8588
Epoch [2/5], Step [8000/12500], Loss: 2.1901
Epoch [2/5], Step [10000/12500], Loss: 2.1492
Epoch [2/5], Step [12000/12500], Loss: 1.8606
Epoch [3/5], Step [2000/12500], Loss: 1.5159
Epoch [3/5], Step [4000/12500], Loss: 1.3586
Epoch [3/5], Step [6000/12500], Loss: 1.6538
Epoch [3/5], Step [8000/12500], Loss: 1.4715
Epoch [3/5], Step [10000/12500], Loss: 1.4329
Epoch [3/5], Step [12000/12500], Loss: 1.0962
Epoch [4/5], Step [2000/12500], Loss: 1.1129
Epoch [4/5], Step [4000/12500], Loss: 1.7177
Epoch [4/5], Step [6000/12500], Loss: 2.3615
Epoch [4/5], Step [8000/12500], Loss: 1.2813
Epoc